In [7]:
import os
import re
import pymupdf
import django_setup
from dotenv import load_dotenv
from IPython.display import Markdown, display

load_dotenv()

gemini_api_key = os.getenv("GEMINI_API_KEY")

In [8]:
import google.generativeai as genai
genai.configure(api_key=gemini_api_key)


In [9]:
def get_model_configured():
    generation_config = {
      "temperature": 0.1,
      "top_p": 0.95,
      "top_k": 40,
      #"max_output_tokens": 8192,
      "response_mime_type": "application/json",
    }

    model = genai.GenerativeModel(
      model_name="gemini-2.0-flash-exp",
      generation_config=generation_config,
    )

    return model

model = get_model_configured()


In [10]:
def clean_text(text, _normalizes=None):
    txt = text
    try:
        normalizes = _normalizes or (
            ('\n ?\d+ ?/ ?\d+ ?\n', '\n'),
            ('[ ]{2,}', ' '),
            (' \\n', '\n'),
            #('([^\.]|\S)\n(\S)', r'\1 \2'),
            ('(\w)\n(.)', r'\1 \2'),
            ('(,)\n(.)', r'\1 \2'),
            #('()\n()', r'\1 \2'),
            ('\n\n', '\n'),
            ('-\n', '-'),
            ('^\n', ''),
            ('–', '-'),
            ('•', '*'),
            ('[“”]', '"'),
            #('', ''),
            #('', ''),
            #('', ''),
            #('', ''),
            #('', ''),
        )

        for regex, new in normalizes:
            search = re.search(regex, text)
            while search:
                text = re.sub(regex, new, text)
                search = re.search(regex, text)
        return text
    except:
        return txt


In [23]:
def make_prompt(text):
    prompt = ("""Analise <CONTEXTO></CONTEXTO> e escreva um json com as seguintes chaves:
              "temas","autores", "destinatarios", "possiveis_destinatarios", "localidades" e "analise".
              Para preencher as chaves, responda as perguntas abaixo:
1) Dentro das diversas áreas da gestão pública municipal, o texto em análise aborda a importância e faz solicitações para a sociedade. Classifique o texto criando uma lista temática consisa e coloque sua resposta à esta pergunta na chave "temas".
2) O texto em análise apresenta um ou mais autores que defendem a importância de uma ou mais ações para a sociedade. Identifique esses autores de forma objetiva, ou seja, nome e cargo. Coloque sua resposta à esta pergunta na chave "autores" no formato de lista, mesmo que seja idetificado apenas um autor.
3) Se o texto em análise apresentar um ou mais destinatários das ações defendidas pelos autores. Identifique esses destinatários com seu nome, cargo e instituição a qual pertence. Explique a importância das ações defendidas pelos autores para esses destinatários no contexto de seus cargos e instituições. Coloque sua resposta à esta pergunta na chave "destinatarios" no formato de lista, mesmo que seja idetificado apenas um destinatário.
4) Com base nas diversas áreas da gestão pública municipal, formule uma lista de possíveis instituições destinatárias não citadas pelos autores e justifique. Coloque sua resposta à esta pergunta na chave "possiveis_destinatarios" no formato de lista.
5) Caso o texto em análise apresente localidades, identifique essas localidades, sejam elas logradouros públicos, bairros ou mesmo prédios públicos e/ou privados, adicionando o tipo identificado. Coloque sua resposta à esta pergunta na chave "localidades" no formato de lista.
6) Formule a defesa do texto analisado com no máximo 200 palavras sobre a importância das ações defendidas pelos autores para os destinatários identificados. É muito importante que escreva essa defesa com um texto leve de forma que uma pessoa leiga no assunto entenda. Coloque sua resposta à esta pergunta na chave "analise" no formato de markdown.

<CONTEXTO>{context}</CONTEXTO>
    """).format(context=text)

    while '  ' in prompt:
        prompt = prompt.replace('  ', ' ')

    return prompt


In [24]:
from sapl.materia.models import MateriaLegislativa

materias = MateriaLegislativa.objects.filter(id=21501, tipo_id=3)

for materia in materias[:1]:
    doc = pymupdf.open(materia.texto_original.path)
    text = ' '.join([page.get_text() for page in doc])
    text = clean_text(text)

prompt = make_prompt(text)

answer = model.generate_content(prompt)

In [25]:
import json

try:
    data = {}
    data['id'] = materia.id
    data['epigrafe'] = str(materia)
    data['ementa'] = materia.ementa
    data.update(json.loads(answer.text))
    data_json = json.dumps(data, indent=4, ensure_ascii=False)
    open(f'data/002_output_{materia.numero}.json', 'w').write(data_json)
except:
    print('Erro ao converter json')
    print(answer.text)
    open('data/002_output.txt', 'w').write(answer.text)